In [4]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score

In [ ]:
dataset = pd.read_csv('./datasets/glcm_135.csv', delimiter="\t").sample(frac=1)

In [ ]:
dataset.head()

In [206]:
dataset = pd.read_csv('./datasets/glrlm_0_sep_diseases.csv', sep=',', index_col=0).sample(frac=1)
dataset = dataset[dataset['isPatho'] <= 1]
X = dataset.iloc[:, :-1]
Y = dataset.iloc[:, -1]

seed = 123
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))



Accuracy: 96.00%


In [237]:
from sklearn.neighbors import KNeighborsClassifier
dataset = pd.read_csv('./datasets/glrlm_0_sep_diseases.csv', sep=',', index_col=0).sample(frac=1)
dataset = dataset[dataset['isPatho'] <= 1]

X = dataset.iloc[:, :-1]
Y = dataset.iloc[:, -1]
seed = 123
test_size = 0.4
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
neig = KNeighborsClassifier(n_neighbors=5)
neig.fit(X_train, y_train)
prob = neig.predict_proba(X_test)
pred = neig.predict(X_test)
# print(prob)
# print(pred)

accuracy = accuracy_score(y_test, neig.predict(X_test))
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 93.33%


In [240]:
from sklearn import svm
dataset = pd.read_csv('./datasets/glrlm_0_sep_diseases.csv', sep=',', index_col=0).sample(frac=1)
dataset = dataset[dataset['isPatho'] <= 1]

X = dataset.iloc[:, 1:-1]
Y = dataset.iloc[:, -1]
seed = 123
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)
accuracy = accuracy_score(y_test, clf.predict(X_test))
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 86.67%


In [242]:
from sklearn import svm
dataset = pd.read_csv('./datasets/glrlm_0_sep_diseases.csv', sep=',', index_col=0).sample(frac=1)
dataset = dataset[dataset['isPatho'] <= 1]

X = dataset.iloc[:, 1:-1]
Y = dataset.iloc[:, -1]
seed = 123
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
accuracy = accuracy_score(y_test, clf.predict(X_test))
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 93.33%


In [ ]:
predicted = clf.predict(X_test)

def calc_accuracy(predicted, y_test):
    if len(predicted) != len(y_test):
        print("Function parameters must be the same size!")
        return

    true_answers = sum(int(predicted[i] == y_test[i]) for i in range(len(y_test)))
    
    return float(true_answers/float(len(y_test)))

calc_accuracy(predicted, y_test.values)


In [1]:
import pandas as pd
from sklearn import svm
from sklearn.model_selection import train_test_split
dataset = pd.read_csv('./datasets/glrlm_0_unit.csv', sep=',', index_col=0).sample(frac=1)

seed = 123
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
svc = svm.SVC(kernel='linear', gamma=.004, C=100)
svc.fit(X_train, y_train)

NameError: name 'X' is not defined

In [ ]:
dataset = pd.read_csv('./datasets/glrlm_0_unit.csv', sep=',', index_col=0).sample(frac=1)
X = dataset.iloc[:, 1:-1]
Y = dataset.iloc[:, -1]

seed = 123
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

In [ ]:
clf = svm.SVC(gamma='scale')
clf.fit(X, y)
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train) 

In [ ]:
X_train